Code for basic LLM

In [2]:
from langchain_groq import ChatGroq

llm= ChatGroq(
    temperature = 0,
    groq_api_key= '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',
    model_name = "llama-3.1-70b-versatile"
)
response= llm.invoke("The forst person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


Loading Web Page

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.bain.com/careers/find-a-role/position/?jobid=10401")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






Associate Consultant Intern






























































Skip to Content











        
        
        Offices
    




 Close 


Offices



North & Latin America




Atlanta


Austin


Bogota


Boston


Buenos Aires


Chicago


Dallas


Denver


Houston


Los Angeles


Mexico City


Minneapolis


Monterrey


New York


Rio de Janeiro


San Francisco


Santiago


São Paulo


Seattle


Silicon Valley


Toronto


Washington, DC






Europe & Africa




Amsterdam


Athens


Berlin


Brussels


Copenhagen


Dusseldorf


Frankfurt


Helsinki


Istanbul


Johannesburg


Kyiv


Lisbon


London


Madrid


Milan


Munich


Oslo


Paris


Rome


Stockholm


Vienna


Warsaw


Zurich




Middle East




Doha


Dubai


Riyadh






Asia & Australia




Bangkok


Beijing


Bengaluru


Ho Chi Minh City


Hong Kong


Jakarta


Kuala Lumpur


Manila


Melbourne


Mumbai


New Delhi


Perth


Seoul


Shanghai


Singapore


Sydney


Tokyo







See all office

Extracting Key Information 

In [4]:
from langchain_core.prompts import PromptTemplate
PromptTemplate = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from carrer's page of a website.
    Your job is to extract the job posting and return them in JSON format constraint
    following keys: 'role', 'experience', 'skills' and 'description'.
    Only return valid JSON. 
    ### VALID JASON (NO PREAMBLE):
)
"""
)
chain_extract = PromptTemplate | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)   

```json
{
  "role": "Associate Consultant Intern",
  "experience": "Internship",
  "skills": [
    "Analytical skills",
    "Interpersonal skills",
    "Strong academic background",
    "High motivation levels"
  ],
  "description": "Our internship program for current students provides broad exposure to the consulting industry and teaches business strategy through full-time immersion on a Bain case team. As an Associate Consultant Intern (ACI), you'll begin with a week of intensive training to learn core consulting skills. You’ll work alongside Bain knowledge experts tackling complex business challenges and use your skills to help drive change in a range of organizations from global leaders to start-ups."
}
```


Getting JSON Output

In [5]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

{'role': 'Associate Consultant Intern', 'experience': 'Internship', 'skills': ['Analytical skills', 'Interpersonal skills', 'Strong academic background', 'High motivation levels'], 'description': "Our internship program for current students provides broad exposure to the consulting industry and teaches business strategy through full-time immersion on a Bain case team. As an Associate Consultant Intern (ACI), you'll begin with a week of intensive training to learn core consulting skills. You’ll work alongside Bain knowledge experts tackling complex business challenges and use your skills to help drive change in a range of organizations from global leaders to start-ups."}


Loading Multiple Portfolio Links in CSV Format 

In [6]:
import pandas as pd 
df = pd.read_csv("Portfolio.csv")
df.

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


Creating a Vector Database and Storing Portfolio.csv

In [10]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')#Creating database on disk so used PersistentClient instead of just Client
collection = client.get_or_create_collection(name  = "Portfolio")
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\pahwa\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████| 79.3M/79.3M [05:26<00:00, 254kiB/s]


In [19]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [21]:
job

{'role': 'Associate Consultant Intern',
 'experience': 'Internship',
 'skills': ['Analytical skills',
  'Interpersonal skills',
  'Strong academic background',
  'High motivation levels'],
 'description': "Our internship program for current students provides broad exposure to the consulting industry and teaches business strategy through full-time immersion on a Bain case team. As an Associate Consultant Intern (ACI), you'll begin with a week of intensive training to learn core consulting skills. You’ll work alongside Bain knowledge experts tackling complex business challenges and use your skills to help drive change in a range of organizations from global leaders to start-ups."}

In [22]:
job = json_res
job['skills']

['Analytical skills',
 'Interpersonal skills',
 'Strong academic background',
 'High motivation levels']

In [24]:
prompt_email= PromptTemplate.from_template( """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Karan, a business development executive at XYZ. XYZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
        Remember you are Karan, BDE at XYZ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
                                          )
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description" : str(job), "link_list" : links})
print(res.content)

Subject: Unlock the Power of AI & Software Consulting for Your Business Needs

Dear Hiring Manager,

I came across the Associate Consultant Intern role at Bain and was impressed by the company's commitment to driving business strategy through innovative solutions. As a Business Development Executive at XYZ, I'd like to introduce you to our expertise in AI & Software Consulting, which can help fulfill your needs in the consulting industry.

At XYZ, we specialize in facilitating seamless integration of business processes through automated tools, empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has a proven track record of delivering high-quality solutions that meet the unique needs of our clients.

I'd like to highlight some of our notable projects that demonstrate our capabilities:

* Machine Learning and Python expertise: https://example.com/ml-python-portfolio
* Android 